In [3]:
import pandas as pd
import seaborn as sns
import matplotlib as plt

In [4]:
df_df_1_org = pd.read_csv(r"..\..\data\raw\Digital_Footprints_pt1.txt")
df_df_2_org = pd.read_csv(r"..\..\data\raw\Digital_Footprints_pt2.txt")
df_cp_org = pd.read_csv(r"..\..\data\raw\Client_Profiles.txt")
df_er_org = pd.read_csv(r"..\..\data\raw\Experiment_Roster.txt")

In [23]:
#concat both footprint csvs for a whole set and dropping duplicate rows
df_df = pd.concat([df_df_1_org, df_df_2_org], axis=0, ignore_index=True)
df_df = df_df.drop_duplicates()

#sorting the date to assure the valid process_step is always the first to shown; valid is defined as "most recent"
df_df.head()
df_df_sorted_date = df_df.sort_values(by=["client_id", "visitor_id", 'date_time'], ascending=True)

print("\n", df_df.shape)
df_df.head(20)

#quick check if all entries are within the duration of experiment
min(df_df["date_time"])
max(df_df["date_time"])




 (744641, 5)


'2017-06-20 23:59:57'

In [36]:
"""
input: dataframe
output: dataframe

function to ONLY take the latest date_time row for EACH process_step PER client_id

"""

def validating_steps(df):
    df['date_time'] = pd.to_datetime(df['date_time'])
    process_steps = ['confirm', 'step_3', 'step_2', 'step_1', 'start']

    df_filtered = df[df['process_step'].isin(process_steps)]

    latest_rows = df_filtered.loc[df_filtered.groupby(['client_id', 'process_step'])['date_time'].idxmax()]
    latest_rows = latest_rows.sort_values(by="date_time", ascending = True)
    
    return latest_rows

#counting the number of valid process_steps to see if the filtering might had issues: 
#start > step_1 > step_2 > step_3 > confirm
valid_df_total = validating_steps(df_df)
valid_df_total["process_step"].value_counts(), valid_df_total.shape


(process_step
 start      118980
 step_1     104978
 step_2      96943
 step_3      91076
 confirm     81145
 Name: count, dtype: int64,
 (493122, 5))

In [32]:
df_cp = df_cp_org.copy()
df_cp.head()

client_id  clnt_tenure_yr  clnt_tenure_mnth  clnt_age gendr  num_accts  \
0     836976             6.0              73.0      60.5     U        2.0   
1    2304905             7.0              94.0      58.0     U        2.0   
2    1439522             5.0              64.0      32.0     U        2.0   
3    1562045            16.0             198.0      49.0     M        2.0   
4    5126305            12.0             145.0      33.0     F        2.0   

         bal  calls_6_mnth  logons_6_mnth  
0   45105.30           6.0            9.0  
1  110860.30           6.0            9.0  
2   52467.79           6.0            9.0  
3   67454.65           3.0            6.0  
4  103671.75           0.0            3.0

In [50]:
#applying the filter of valid rows to client profile dataframe

df_cp_valid = pd.merge(valid_df_total, df_cp, how="left", on="client_id")
df_cp_valid

client_id             visitor_id                      visit_id  \
0         7179755  167765295_97487764427   264484508_5982901710_928530   
1         7179755  167765295_97487764427   264484508_5982901710_928530   
2         7179755  167765295_97487764427   264484508_5982901710_928530   
3         7179755  167765295_97487764427   264484508_5982901710_928530   
4         7179755  167765295_97487764427   264484508_5982901710_928530   
...           ...                    ...                           ...   
493117    4159912  169591677_18042085072  358114812_31313949217_790854   
493118    4159912  169591677_18042085072  358114812_31313949217_790854   
493119    4159912  169591677_18042085072  358114812_31313949217_790854   
493120    5640210  854970757_75895258102  959123368_90077952691_695747   
493121    4227031   375803585_1924749079  694062705_73082073487_259240   

       process_step           date_time  clnt_tenure_yr  clnt_tenure_mnth  \
0             start 2017-03-15 00:19:28            28.0             343.0   
1            step_1 2017-03-15 00:20:50            28.0             343.0   
2            step_2 2017-03-15 00:22:52            28.0             343.0   
3            step_3 2017-03-15 00:23:47            28.0             343.0   
4           confirm 2017-03-15 00:26:24            28.0             343.0   
...             ...                 ...             ...               ...   
493117        start 2017-06-20 23:58:40             NaN               NaN   
493118       step_1 2017-06-20 23:59:00             NaN               NaN   
493119       step_2 2017-06-20 23:59:23             NaN               NaN   
493120        start 2017-06-20 23:59:53             NaN               NaN   
493121        start 2017-06-20 23:59:57             NaN               NaN   

        clnt_age gendr  num_accts        bal  calls_6_mnth  logons_6_mnth  
0           67.5     M        3.0  352322.35           2.0            5.0  
1           67.5     M        3.0  352322.35           2.0            5.0  
2           67.5     M        3.0  352322.35           2.0            5.0  
3           67.5     M        3.0  352322.35           2.0            5.0  
4           67.5     M        3.0  352322.35           2.0            5.0  
...          ...   ...        ...        ...           ...            ...  
493117       NaN   NaN        NaN        NaN           NaN            NaN  
493118       NaN   NaN        NaN        NaN           NaN            NaN  
493119       NaN   NaN        NaN        NaN           NaN            NaN  
493120       NaN   NaN        NaN        NaN           NaN            NaN  
493121       NaN   NaN        NaN        NaN           NaN            NaN  

[493122 rows x 13 columns]

In [44]:
# importing experiment roster and removing invalid rows (no group assigned)

df_er = df_er_org.copy()
df_er = df_er.dropna()
df_er.head(), df_er.shape

(   client_id Variation
 0    9988021      Test
 1    8320017      Test
 2    4033851   Control
 3    1982004      Test
 4    9294070   Control,
 (50500, 2))

In [94]:
#seperating the total valid df into testing and control group
testing = df_er[df_er["Variation"] == "Test"]
testing = pd.merge(testing, df_cp_valid, how="left", on="client_id")
testing.columns = testing.columns.str.lower().str.lstrip()

control = df_er[df_er["Variation"] == "Control"]
control = pd.merge(control, df_cp_valid, how="left", on="client_id")
control.columns = control.columns.str.lower().str.lstrip()



control.variation.unique()
control.shape
control.head(30)

client_id variation             visitor_id                      visit_id  \
0     4033851   Control  762728880_76361333336  949661017_22392791362_127391   
1     4033851   Control  762728880_76361333336  949661017_22392791362_127391   
2     4033851   Control  762728880_76361333336  949661017_22392791362_127391   
3     4033851   Control  762728880_76361333336  949661017_22392791362_127391   
4     4033851   Control  762728880_76361333336  949661017_22392791362_127391   
5     9294070   Control  489274391_63794562906  116640698_27328181095_188045   
6     6651403   Control  387438258_49845138949  754106292_61370647593_160929   
7     6651403   Control  387438258_49845138949  754106292_61370647593_160929   
8     6651403   Control  387438258_49845138949  754106292_61370647593_160929   
9     6651403   Control  387438258_49845138949   15735650_17000221458_175134   
10    6651403   Control  387438258_49845138949   15735650_17000221458_175134   
11    2105948   Control  111894218_79248359000  284161492_75966778593_698992   
12    2105948   Control  111894218_79248359000  284161492_75966778593_698992   
13    2105948   Control  111894218_79248359000  284161492_75966778593_698992   
14    2105948   Control  111894218_79248359000   400183608_67913149333_45420   
15    2105948   Control  111894218_79248359000   400183608_67913149333_45420   
16    9814849   Control  643702528_85907162029  912594178_11107066778_946103   
17    9814849   Control  643702528_85907162029  912594178_11107066778_946103   
18    9814849   Control  643702528_85907162029  912594178_11107066778_946103   
19    9814849   Control  643702528_85907162029  912594178_11107066778_946103   
20    9814849   Control  643702528_85907162029  912594178_11107066778_946103   
21    7286739   Control  450196013_75056881844  841652923_56424358687_156740   
22    7286739   Control  450196013_75056881844  841652923_56424358687_156740   
23    7286739   Control  450196013_75056881844  841652923_56424358687_156740   
24    7286739   Control  450196013_75056881844  841652923_56424358687_156740   
25    7286739   Control  450196013_75056881844  841652923_56424358687_156740   
26    4446011   Control  863538055_51523102542  607683472_36133221698_258790   
27    4446011   Control  863538055_51523102542  607683472_36133221698_258790   
28    4446011   Control  863538055_51523102542  607683472_36133221698_258790   
29    4446011   Control  863538055_51523102542  607683472_36133221698_258790   

   process_step           date_time  clnt_tenure_yr  clnt_tenure_mnth  \
0         start 2017-04-05 12:24:04            12.0             149.0   
1        step_1 2017-04-05 12:24:12            12.0             149.0   
2        step_2 2017-04-05 12:24:43            12.0             149.0   
3        step_3 2017-04-05 12:26:08            12.0             149.0   
4       confirm 2017-04-05 12:29:03            12.0             149.0   
5         start 2017-04-17 11:16:28             5.0              70.0   
6        step_2 2017-04-04 17:33:55            13.0             162.0   
7        step_3 2017-04-04 17:34:03            13.0             162.0   
8       confirm 2017-04-04 17:38:39            13.0             162.0   
9         start 2017-04-07 02:37:12            13.0             162.0   
10       step_1 2017-04-07 02:37:23            13.0             162.0   
11       step_2 2017-04-05 13:59:46             6.0              74.0   
12       step_3 2017-04-05 14:00:00             6.0              74.0   
13      confirm 2017-04-05 14:04:56             6.0              74.0   
14        start 2017-04-06 10:19:49             6.0              74.0   
15       step_1 2017-04-06 10:20:18             6.0              74.0   
16        start 2017-04-05 11:21:53            12.0             149.0   
17       step_1 2017-04-05 11:22:00            12.0             149.0   
18       step_2 2017-04-05 11:22:08            12.0             149.0   
19       step_3 2017-04-05 11:23:58            12.0            

In [ ]:
testing = pd.merge(df_er, df_cp_valid, how="left", on="client_id")
testing